<a href="https://colab.research.google.com/github/Mani1213/LLM-s/blob/main/desc2titles_llama_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
from datasets import load_dataset,DatasetDict,Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

2023-10-25 07:54:59.101914: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-25 07:54:59.170009: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-25 07:54:59.170057: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-25 07:54:59.170225: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-25 07:54:59.191005: I tensorflow/core/platform/cpu_feature_g

In [ ]:
df = pd.read_parquet("desc2title_data.parquet")
df = df.sample(100000,random_state=45)

In [ ]:
df=df[df.cleaned_title.apply(lambda x:isinstance(x,str))]

In [ ]:
df['text']=df.apply(lambda x:"<s>[INST] Generate multiple relevant jobtitles for the description below ?\n"+x['description']+"[/INST] "+x['cleaned_title']+" </s>",axis=1)

/tmp/ipykernel_279411/1108369352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text']=df.apply(lambda x:"<s>[INST] Generate multiple relevant jobtitles for the description below ?\n"+x['description']+"[/INST] "+x['cleaned_title']+" </s>",axis=1)


In [ ]:
df=df[['text']]

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"

lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = True

output_dir = "./results"
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 8

# Batch size per GPU for evaluation
per_device_eval_batch_size = 8

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 15

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 2048

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
dataset=Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 99997
})

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,

)

# Train model
trainer.train()

# Save trained model


Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/mlteam/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/mlteam/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Special tokens have been added in the vocabulary, make sur

Map:   0%|          | 0/99997 [00:00<?, ? examples/s]

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/mlteam/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
15,2.125000
30,2.320400
45,2.520200
60,2.400300


In [ ]:
new_model="results/desc2title_llama"
trainer.model.save_pretrained(new_model)
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

In [ ]:
from transformers import pipeline

In [ ]:
pipe= pipeline(task="text-generation", model="results/desc2title_llama", tokenizer="results/desc2title_llama", max_length=350,use_fast=True,device_map = {"": 0},max_new_tokens=350,)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/mlteam/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/mlteam/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
prompt= "Generate 30 relevant jobtitles for the description below ?\n Warehouse ReceiverPay rate: Up to $18.40Job Summary!You will become part of the team at the Amazon warehouse that takes care of the large items&mdash;the items that customer orders and weigh over 50 pounds, like furniture, big-screen TVs, appliances, and more. You will get the necessary training to use the latest technology to move these heavy bulk products. Your non-experience in handling these items will not be a barrier to gaining valuable job inputs, and you will be training on powered industrial trucks (PIT) such as pallet drivers and forklifts. There will be shifts where you have to manage our power equipment to get the customer orders ready; at other times, you will be a part of our warehouse team at fulfillment centers or delivery stations handling those hands-on items that require expert service before they are sent for delivery. If you are someone who wants to become a heavy equipment operator, this is the opportunity for you.What are the duties and responsibilities of the job?The job duties include:You need to operate powered industrial trucks (PITs). Training on equipment and certification shall be given free of cost.Follow the on-screen prompts for certain tasks and work as per directions.Take count of the new bulk and heavy inventory and put them away.Load the trucks with order packages for shipment.Use technology like smartphones and computers for processing orders.Work and operate such machinery that reaches a height of 35 ft.Additional responsibilities would include:Lifting weights up to 49 pounds.During shifts, use hand trucks, dollies, carts, and other gears to move things around.Stand, walk, squat, push, pull, bend, and reach during your shifts.Going up and down the stairs wherever applicable.Follow the safety standards to handle products that weigh more than 50 pounds.What is the vibe at an Amazon warehouse?Noise: The place can get noisy because of different warehouse activities. You will get hearing protection gears from us if you need them.Surroundings: Your surrounding will be filled with stand-up forklifts, order pickers, turret trucks, moving machines, and mobile carts.Activity: Your job might make you stand in one place for a long time, climb stairs, or walk around.Dress code: There is no dress code. However, you will have to follow a few rules for safety measures. You will have to wear comfortable, closed-toe shoes. Based on the job location or the role, Amazon will provide you with a $110 Zappos gift code for purchasing shoes to prepare you for your first day on the job.Temperature: You will have to work in extreme temperatures. Though there are climate control measures inside the warehouse, temperature could vary between 60&deg;F and 90&deg;F in some parts of the warehouse. During summers, the temperature can exceed 90&deg;F inside trailers or the truck yard.Safety: Safety is a top priority for all of us. Every employee is provided with appropriate safety gear, and the teams ensure safety tips are shared on a regular basis.Why will you want to do this job?Team environment: At Amazon, the team environment is very supportive; team members support one another, even if it is a small warehouse or a big one.Shift flexibility: You can enjoy a flexible schedule. You can take your shifts in the early morning to shift in the daytime, evenings, or even overnight and weekend shifts. There are options for three-day shifts and four-day shifts. You can also opt for anytime shifts for as little as 4 hours a week.Schedule options:There are multiple work schedule options to choose from. Based on your preference, you can work full-time (40 hours a week), reduced time (30 to 36 hours a week), or part-time (20 hours or less). You also have the option of working overtime.Free training: You will learn how to drive power industrial trucks like pallet drivers or forklifts and how to use other advanced Amazon technologies, including working with robots.Career advancement: We as a company believe in upgrading our employee skills with various career development programs, including training and upscaling programs. We also provide tuition reimbursement to a select group of employees.Stay occupied: Such is your job that it will keep you and your team engaged throughout the shift hours.New skills: Based on your role and location, you will learn how to use the latest Amazon technologies, including robotics and handheld gadgets.Why is Amazon so preferred?Many of our jobs come with loads of benefits, including career development programs, health benefits, parental leaves, and ways to save for the future. We believe in an inclusive and safe working ecosystem. Amazon is considered one of the best places to work in the world. Our offerings may vary as per the job role, location, and work schedule. Some of the benefits include:401(k) savings option.Employee Assistance Program.Competitive wages are paid weekly.Healthcare includes medical, optical, dental benefits and prescribed drugsPaid time off (PTO).Holiday pay options.Skill development and training while on the job.Employee discounts on various items and restaurants.Minimum life insurance.AD&amp;D insurance.Company-supported long-term and short-term disability benefits.On-the-job training and skill enhancement.Employee Assistance Program.Maternity and parental leaves.Adoption assistance.Learn more about Amazon Benefits and Culturehttps://hiring.amazon.com/why-amazon/benefits#/https://hiring.amazon.com/why-amazon/culture#/Requirements:Candidates must be 18 years of age or older with the ability to acknowledge and comply with the job requirements and safety guidelines.How to apply?You can begin by applying here. We can help you with the application process, or learn more about how hiring happens.For details, visit https://hiring.amazon.com/hiring-process#/.If you have a disability and need accommodation during the application and hiring process, including support for the New Hire Event, or need to initiate a request prior to starting your Day 1, please visit https://hiring.amazon.com/people-with-disabilities#/ or contact the Applicant-Candidate Accommodation Team (ACAT). You can reach us by phone at 888-435-9287, Monday through Friday, between 6 a.m. and 4 p.m. PT.Amazon is committed to a diverse and inclusive workplace. Amazon is an equal opportunity employer and does not discriminate on the basis of race, national origin, gender, gender identity, sexual orientation, protected veteran status, disability, age, or another legally protected status."

In [ ]:
%%time
des=pipe(f"<s>[INST] {prompt} [/INST]")[0]['generated_text']

Both `max_new_tokens` (=350) and `max_length`(=350) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CPU times: user 12.3 s, sys: 147 ms, total: 12.5 s
Wall time: 12.5 s


In [ ]:
idx=des.index('[/INST]')+7
des=des[idx:]
des.split("\n")[2:-2]

['1. Warehouse Receiver',
 '2. Inventory Specialist',
 '3. Bulk Receiver',
 '4. Heavy Equipment Operator',
 '5. Powered Industrial Truck Operator',
 '6. Forklift Driver',
 '7. Warehouse Worker',
 '8. Order Fulfillment Specialist',
 '9. Shipping and Receiving Clerk',
 '10. Inventory Control Specialist',
 '11. Supply Chain Coordinator',
 '12. Warehouse Technician',
 '13. Material Handler',
 '14. Equipment Operator',
 '15. Shipping Supervisor',
 '16. Receiving Manager',
 '17. Inventory Lead',
 '18. Warehouse Lead',
 '19. Logistics Coordinator',
 '20. Supply Chain Analyst',
 '21. Warehouse Coordinator',
 '22. Inventory Analyst',
 '23. Shipping and Receiving Supervisor',
 '24. Supply Chain Manager',
 '25. Warehouse Operations Manager',
 '26. Inventory Planner',
 '27. Logistics Coordinator',
 '28. Shipping Manager',
 '29. Receiving Manager',
 '30. Warehouse Manager']